## Imports

In [1]:
#from __future__ import print_function, division

import torch
import torch.nn.parallel
import torch.utils
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
from PIL import Image
import DrawingWithTensors

from torchvision.transforms import ToPILImage
#from IPython.display import Image
#to_img = ToPILImage()
#from IPython.display import Image

#plt.ion()   # interactive mode

#original code for training: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

#imports related to fully convolutional network
import torchfcn

#original paths for FCNs:
#/home/peo5032/data/models/chainer/fcn16s_from_caffe.npz
# calling torchfcn.models.FCN16s.pretrained_model yields:
# might need to call download on it first: torchfcn.models.FCN16s.download()
#'/home/peo5032/data/models/pytorch/fcn16s_from_caffe.pth'

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os
import argparse

# initiate the parser
parser = argparse.ArgumentParser(description = "List of options to run application when creating custom datset")

parser = argparse.ArgumentParser()  
parser.add_argument("-V", "--version", help="show program version", action="store_true")
parser.add_argument("-w", "--weights", help="full path to save weights")

_StoreAction(option_strings=['-w', '--weights'], dest='weights', nargs=None, const=None, default=None, type=None, choices=None, help='full path to save weights', metavar=None)

In [3]:
PRETRAINED_PATH = '/home/peo5032/data/models/pytorch/fcn16s_from_caffe.pth'
LOAD_LOCATION = "/home/peo5032/Documents/COMP594/model.pt"

NUM_CLASSES = 7
EPOCHS = 4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu" #just for testing for sunlab
imageSize = 400
batchSize = 1
iteration = "1"
newTraining = False

#change values if user specifies non-default values
args = parser.parse_args(["-w", "/home/peo5032/Documents/COMP594/model.pt"])

# check for --version or -V
if args.version:  
    print("this is version 0.1")

    
if args.weights:
    print("load location is set to", args.weights)
    LOAD_LOCATION = args.weights
    

#TODO in arguments
# root folder location
# saved weights location

load location is set to /home/peo5032/Documents/COMP594/model.pt


In [4]:
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

## Load Data

In [5]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
data_dir = '/home/peo5032/Documents/COMP594/input/gen'+iteration
dataset = ImageFolderWithPaths(data_dir, transform=data_transforms) # our custom dataset
dataloaders = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle=True, num_workers=1)


new_road_factory = DrawingWithTensors.datasetFactory()

## Training Routine without Validation Steps

In [6]:
def showInferenceOnImage(img, tensor, class_label, threshold, classMap):
    IMAGE_SIZE = 400
    imgTMP = img.copy()
    imgMap = imgTMP.load()
    class_type_corresponding_channel = classMap[class_label]
    print("index for channel", class_label, ":", class_type_corresponding_channel)    
    for i in range(0, IMAGE_SIZE):
        for j in range(0, IMAGE_SIZE):
            if tensor[class_type_corresponding_channel, i,j] >= threshold:
                #hide non-class label in black
                imgMap[i,j] = (0,0,0)
        
    return imgTMP

## Load Pretrained Model Weights

In [7]:
model = torch.load(LOAD_LOCATION)
model = model.to(device)
model.eval()

FCN16s(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(100, 100))
  (relu1_1): ReLU(inplace)
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace)
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace)
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace)
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,

In [8]:
TEST_LOCATION_ROOT = "/home/peo5032/Documents/COMP594/input/gen1/val"
#img = Image.open(TEST_LOCATION_ROOT+"/1.png")
#img2 = Image.open(TEST_LOCATION_ROOT+"/7.png")
#inputs = torch.zeros(2,3,400,400)
#inputs[0] = data_transforms(img)
#inputs[1] = data_transforms(img2)

In [9]:
#outputs = model(inputs)

In [10]:
#img

In [11]:
#outputs[0][1]

In [12]:
new_road_factory.classMap

{'background': 0,
 'left-shoulder': 1,
 'left-yellow-line-marker': 2,
 'white-lane-markers': 3,
 'lane': 4,
 'right-white-line-marker': 5,
 'right-shoulder': 6}

In [13]:
#eval_class = "white-lane-markers"
#min_score =  torch.min(outputs[0][new_road_factory.classMap[eval_class]]).item()
#max_score = torch.max(outputs[0][new_road_factory.classMap[eval_class]]).item()
#middle_score = (min_score + max_score)/2
#threshold = (middle_score + (0.10 * middle_score))
#print("mind is", min_score)
#print("max is", max_score)
#print("threshold is", threshold)
#showInferenceOnImage(img, outputs[0], eval_class, middle_score + (0.10 * middle_score) , new_road_factory.classMap)

In [14]:
criterion = torch.nn.BCELoss()
criterion2 = torch.nn.BCEWithLogitsLoss()

In [19]:
inputs = torch.zeros(1,1,4,4)
outputs = torch.ones(1,1,4,4)


In [20]:
loss = criterion(inputs, outputs)
loss2 = criterion(inputs, outputs)

In [24]:
loss.item()

27.631019592285156

In [18]:
loss2.item()

27.631019592285156